In [ ]:
!pip install langchain neo4j openai tiktoken pytube youtube_transcript_api env

In [1]:
import os

from pytube import Playlist
from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Neo4jVector
from langchain.document_loaders import YoutubeLoader

from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain

In [14]:
# Process all videos from the playlist
playlist_url = "https://www.youtube.com/watch?v=1CqZo7nP8yQ&list=PL9Hl4pk2FsvUu4hzyhWed8Avu5nSUXYrb"
playlist = Playlist(playlist_url)
video_ids = [_v.split('v=')[-1] for _v in playlist.video_urls]
print(f"Processing {len(videos)} videos.")

In [15]:
# Init text splitter with chunk size 512 (https://www.pinecone.io/learn/chunking-strategies/)
text_splitter = TokenTextSplitter.from_tiktoken_encoder(chunk_size=512, chunk_overlap=20)

In [16]:
from env import setup_env
setup_env()

In [17]:
# Read their captions and process it into documents with above defined text splitter
documents = []
for video_id in video_ids:
    try:
      loader = YoutubeLoader(video_id=video_id)
      documents.append(loader.load()[0])
    except: # if there are no english captions
      pass
print(f"Read captions for {len(documents)} videos.")

In [18]:
# Split documents
splitted_documents = text_splitter.split_documents(documents)
print(f"{len(splitted_documents} documents ready to be processed.")

[Document(page_content="I hope the toast is done you may have seen this video of an Olympic track cyclist powering a toaster and today the three of us are going to see just what that feels like 700 WTS for over a minute we've got Dave how much cycling experience do you have uh a couple Pelon rides I'm like a average cyclist and we have EJ over here how would you describe yourself I would say a racer I trained quite a bit he's being modest and how do you describe yourself don't bite drop it all right I have no idea what this going to feel like here we go now you may be wondering how can a stick figure like me be putting out the same amount of power as this Quadzilla the answer is urg mode his bike and mine are currently programmed to lock in at 700 WT give or take the exact amount of energy needed to power a toaster so it doesn't matter if we pedal faster or slower the resistance will automatically adjust to maintain 700 Watts which is lucky for me because I looked up his max Sprint an 

In [19]:
# Contruct vector
neo4j_vector = Neo4jVector.from_documents(
    embedding=OpenAIEmbeddings(),
    documents=splitted_documents,
    url=os.environ['NEO4J_URI'],
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD'],
    search_type="hybrid"
)

In [26]:
# Prepare Q&A object
chat_mem_history = ChatMessageHistory(session_id="1")
mem = ConversationBufferWindowMemory(k=3, memory_key="chat_history", chat_memory=chat_mem_history, return_messages=True)
q = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0.2),
    memory=mem,
    retriever=neo4j_vector.as_retriever(),
    verbose=True,
    max_tokens_limit=4000
)

In [33]:
# Perform Q&A flow - first question
response = q.run('What can you tell me about the GenAI stack?')
response

{'question': 'How much wantts does olimpic cyclists make during the sprint?',
 'chat_history': [HumanMessage(content='How are you?', additional_kwargs={}, example=False),
  AIMessage(content="As an AI, I don't have feelings, so I don't experience emotions like humans do. But thank you for asking! How can I assist you today?", additional_kwargs={}, example=False),
  HumanMessage(content='What is the video about?', additional_kwargs={}, example=False),
  AIMessage(content="The content of the video is a group of individuals attempting to replicate the feat of an Olympic track cyclist who powered a toaster by generating 700 watts of power for over a minute. They discuss their cycling experience and then proceed to ride stationary bikes programmed to lock in at 700 watts. They share their experiences and compare their thigh sizes to the cyclist in the original video. They also mention that one of them, EJ, can generate a significant amount of power and has the best chance of matching the Ol

In [ ]:
# Follow up question that requires previous answers (memory)
response = q.run('Who talked about it?')
response